### An app to help drivers find a spot to park at UT

In [1]:
import sqlite3
from datetime import datetime, timedelta
import random

In [2]:
# Connect to the chinook database
db=sqlite3.connect("/Users/abhil/repo/park-ut/db/UTPark.db")
#db=sqlite3.connect("/Users/Vrinda Sharma/Documents/APAD/Assignments/Project-1/park-ut/db/UTPark.db")

In [3]:
cursor = db.cursor()

def row_to_user (row):
    user = {}
    user['id'] = row[0]
    user['name'] = row[1]
    user['email'] = row[2]
    user['admin'] = bool(row[3])
    return user

def fetch_all():
    all = cursor.fetchall()
    for pl in all:
        print(pl)

def fetch_users():
    admin = None
    non_admin = None
    cursor.execute('''SELECT * FROM users''')
    all = cursor.fetchall()
    for pl in all:
        print(pl)
        if non_admin is None and pl[3] is 0:
            non_admin = row_to_user(pl)
        elif admin is None and pl[3] is 1:
            admin = row_to_user(pl)
            
    return (non_admin, admin)
            
(non_admin, admin) = fetch_users()

print('non_admin: ', non_admin)
print('admin: ', admin)


(1, 'Vrinda', 'vrinda.sha@utexas.edu', 1)
(2, 'Catherine', 'cathy.morgan@utexas.edu', 0)
(3, 'Mac', 'macmiller@gmail.com', 0)
(4, 'Katie', 'Katie@gmail.com', 0)
(5, 'Abhi', 'Abhi@gmail.com', 0)
non_admin:  {'id': 2, 'name': 'Catherine', 'email': 'cathy.morgan@utexas.edu', 'admin': False}
admin:  {'id': 1, 'name': 'Vrinda', 'email': 'vrinda.sha@utexas.edu', 'admin': True}


In [5]:
def add_user(user, name, email, is_admin):
    # add new user
    if not (user['admin']):
        print('Insufficient access.')
        return
    
    try:
        insertsql = '''INSERT INTO users (Name,Email,Admin) VALUES ('{}', '{}', {})'''
        cursor.execute(insertsql.format(name, email, 1 if is_admin is True else False))
        db.commit()
        print('Successfully added a user.')
    except Exception as e:
        print(e)

add_user(non_admin, 'Abhi', 'Abhi@gmail.com', False)
add_user(admin, 'Abhi', 'Abhi@gmail.com', False)

Insufficient access.
UNIQUE constraint failed: users.Email


In [6]:
def add_garages(user, name, address):
    if not (user['admin']):
        print('Insufficient access.')
        return
    
    try:
        insertsql = '''INSERT INTO garages (Name, Address) VALUES ('{}', '{}')'''
        cursor.execute(insertsql.format(name, address))
        db.commit()
        print('Successfully added a garage.')
    except Exception as e:
        print(e)

add_garages(non_admin, 'Guadulupe', 'Guadulupe street')
add_garages(admin, 'Guadulupe', 'Guadulupe street')

Insufficient access.


In [11]:
def add_spots(user, garage_id, level):
    if not (user['admin']):
        print('Insufficient access.')
        return
    
    try:
        insertsql = '''INSERT INTO spots (GarageId, Level, Number) VALUES ({}, {}, {})'''
        cursor.execute(insertsql.format(garage_id, level, random.randint(1,10000)))
        db.commit()
        print('Successfully added a spot.')
    except Exception as e:
        print(e)

add_spots(non_admin, 1, 1)
add_spots(admin, 1, 1)

Insufficient access.


In [7]:
def startevent(user, spotid, onbehalfof=None):
    # this reserves a spot for parking for 10 minutes
    if onbehalfof is not None:
        if not (user['admin']):
            print('Insufficient access to reserve on behalf of another user.')
            return
        user = onbehalfof
    
    now = datetime.now()
    
    try:
        cursor.execute('''SELECT Expiry FROM reservations WHERE SpotId={}'''.format(spotid))
        lastexpiry = max(datetime.fromisoformat(row[0]) for row in cursor.fetchall())

        if (lastexpiry > now):
            print('Reservation already exists.')
            return

        expire = (datetime.now() + timedelta(minutes=10)).isoformat(timespec='minutes')
        insertsql = '''INSERT INTO reservations (SpotId, Expiry) VALUES ({}, '{}')'''
        cursor.execute(insertsql.format(spotid, expire))
        db.commit()
        print('Successfully reserved.')
    except Exception as e:
        print(e)
    
startevent(non_admin, 1)
startevent(non_admin, 1, non_admin)
startevent(admin, 1, non_admin)

Successfully reserved.
Insufficient access to reserve on behalf of another user.
Reservation already exists.


In [17]:
cursor.execute("""Select count(*) from garages""")
fetch_all()

(3,)


In [18]:
cursor.execute("""Select count(*) from spots""")
fetch_all()

(3,)


In [83]:
db.close()